In [1]:
from typing import NamedTuple, Set
import re

In [2]:
def tokenize(text: str) -> Set[str]:
    text = text.lower()                         # Mengubah teks menjadi huruf kecil,
    all_words = re.findall("[a-z0-9']+", text)  # mengekstrak kata-kata, dan
    return set(all_words)                       # mengembalikan set kata unik.

In [5]:
from typing import List, Tuple, Dict, Iterable
import math
from collections import defaultdict

class Message(NamedTuple):
    text: str
    is_spam: bool

class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k
        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0

    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1

            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1

    def _probabilities(self, token: str) -> Tuple[float, float]:
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]
        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)
        return p_token_spam, p_token_ham

    def predict(self, text: str) -> float:
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0

        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)

            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)

        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)

In [12]:
# Data training dari tabel
data_training = [
    ("laki-laki 20 8.000.000 single kendaraan pribadi", False),
    ("laki-laki 35 14.000.000 single kendaraan umum", True),
    ("perempuan 26 10.000.000 single kendaraan umum", True),
    ("perempuan 27 12.000.000 menikah kendaraan pribadi", False),
    ("laki-laki 21 9.000.000 single kendaraan pribadi", False),
    ("laki-laki 22 11.000.000 single kendaraan pribadi", False),
    ("perempuan 32 15.000.000 menikah kendaraan umum", True),
    ("perempuan 26 8.000.000 menikah kendaraan umum", True),
    ("laki-laki 25 9.000.000 single kendaraan umum", True),
    ("perempuan 20 10.000.000 single kendaraan pribadi", False),
]

# Menyusun data training menjadi bentuk pesan (Message)
messages = [Message(text, is_spam) for text, is_spam in data_training]

# Melatih model
model = NaiveBayesClassifier(k=0.5)
model.train(messages)


In [20]:
text_data_11 = "Perempuan 27 12.000.000 Single ?"
text_data_12 = "Laki-Laki 35 14.000.000 Menikah ?"

prediction_11 = model.predict(text_data_11)
prediction_12 = model.predict(text_data_12)

# Menetapkan batasan misalnya jika probabilitas > 0.5, maka kendaraan pribadi; jika probabilitas <= 0.5, maka kendaraan umum.
result_11 = "Kendaraan Pribadi" if prediction_11 > 0.5 else "Kendaraan Umum"
result_12 = "Kendaraan Pribadi" if prediction_12 > 0.5 else "Kendaraan Umum"

print("Prediksi untuk data ke-11:", result_11)
print("Prediksi untuk data ke-12:", result_12)

Prediksi untuk data ke-11: Kendaraan Umum
Prediksi untuk data ke-12: Kendaraan Pribadi
